In [1]:
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.2.1.jar pyspark-shell'  


In [2]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import sys
import json

In [3]:
sc = SparkContext.getOrCreate()
sc.stop()




In [4]:
sc = SparkContext(appName = "PythonStreamingReciever")
ssc = StreamingContext(sc, 5)

In [5]:
kafkaStream = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming',  {'province':1})
lines = kafkaStream.map(lambda x:x[1])
counts = lines.flatMap(lambda line:line.split(" ")).map(lambda word:(word,1)).reduceByKey(lambda a,b:a+b)
counts.pprint()

In [6]:
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers = 'localhost:9092')


In [7]:
def process(rdd):
    print(rdd)
    message = json.dumps(rdd.map(lambda x:[str(x[0]),str(x[1])]).collect())

    producer.send('result',  message.encode('utf-8'))


In [8]:
counts.foreachRDD(process)
ssc.start()

In [ ]:
ssc.awaitTermination()